# 通义千问-基础知识点
## DashScope 是平台，通义千问是模型，DashScope 是提供该模型服务的平台。
可以在 DashScope 平台上调用通义千问的不同版本（如 Qwen-turbo、Qwen-plus、Qwen-max 等），用于文本生成、对话理解、代码编写等任务。

| 名称 | 类型 | 关系 |
|------|------|------|
| **通义千问（Qwen）** | 大型语言模型 | 是 DashScope 平台上提供的核心模型之一 |
| **DashScope** | 大模型服务平台 | 提供通义千问等模型的 API 接口和服务支持 |

你可以把 DashScope 看作是一个“AI 工具箱”，而通义千问就是这个工具箱中最强大的“多功能瑞士军刀”。

In [1]:
# Install the package
%pip install --upgrade --quiet  dashscope

Note: you may need to restart the kernel to use updated packages.


## 获取通义千问API-KEY

地址：https://bailian.console.aliyun.com/console?tab=model#/api-key

In [2]:
# Get a new token: https://help.aliyun.com/document_detail/611472.html?spm=a2c4g.2399481.0.0
from getpass import getpass

DASHSCOPE_API_KEY = getpass()

 ········


In [3]:
import os

os.environ["DASHSCOPE_API_KEY"] = DASHSCOPE_API_KEY

In [4]:
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_core.messages import HumanMessage

chatLLM = ChatTongyi(
    streaming=True,
)
res = chatLLM.stream([HumanMessage(content="hi")], streaming=True)
for r in res:
    print("chat resp:", r)

chat resp: content='Hello' additional_kwargs={} response_metadata={} id='run--9c79e2da-6101-49c3-96fd-18c285d02151'
chat resp: content='!' additional_kwargs={} response_metadata={} id='run--9c79e2da-6101-49c3-96fd-18c285d02151'
chat resp: content=' How' additional_kwargs={} response_metadata={} id='run--9c79e2da-6101-49c3-96fd-18c285d02151'
chat resp: content=' can I assist you' additional_kwargs={} response_metadata={} id='run--9c79e2da-6101-49c3-96fd-18c285d02151'
chat resp: content=' today?' additional_kwargs={} response_metadata={} id='run--9c79e2da-6101-49c3-96fd-18c285d02151'
chat resp: content='' additional_kwargs={} response_metadata={'finish_reason': 'stop', 'request_id': 'f2b1cf39-fca8-9868-85a2-caf456e4250a', 'token_usage': {'input_tokens': 9, 'output_tokens': 9, 'total_tokens': 18, 'prompt_tokens_details': {'cached_tokens': 0}}} id='run--9c79e2da-6101-49c3-96fd-18c285d02151'


In [5]:
from langchain_core.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(
        content="你是一位能把中文翻译成日文的得力助手。"
    ),
    HumanMessage(
        content="将这句话从中文翻译成日文。我喜欢编程。"
    ),
]
chatLLM.invoke(messages)

AIMessage(content='私はプログラミングが好きです。', additional_kwargs={}, response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'stop', 'request_id': 'e9de1e8e-b73d-9184-ad2b-6b2ddc6716f4', 'token_usage': {'input_tokens': 38, 'output_tokens': 8, 'total_tokens': 46, 'prompt_tokens_details': {'cached_tokens': 0}}}, id='run--728ffac9-c7c7-49af-9bcf-079d42362475-0')

## 工具调用（Tool Calling）  
ChatTongyi 支持工具调用 API，该功能允许你描述工具及其参数，并让模型返回一个 JSON 对象，其中包含要调用的工具以及调用该工具所需的输入参数。

In [6]:
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_core.tools import tool


@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int


llm = ChatTongyi(model="qwen-turbo")

llm_with_tools = llm.bind_tools([multiply])

msg = llm_with_tools.invoke("5乘以42等于多少?")

print(msg)

content='' additional_kwargs={'tool_calls': [{'function': {'name': 'multiply', 'arguments': '{"first_int": 5, "second_int": 42}'}, 'index': 0, 'id': 'call_3199a256de3849918fb01b', 'type': 'function'}]} response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'tool_calls', 'request_id': '2fc5beff-8be4-9a56-9356-2360da5cbdcb', 'token_usage': {'input_tokens': 179, 'output_tokens': 25, 'total_tokens': 204, 'prompt_tokens_details': {'cached_tokens': 0}}} id='run--56bcd921-bbd0-4699-9e17-89c81ea9a8ab-0' tool_calls=[{'name': 'multiply', 'args': {'first_int': 5, 'second_int': 42}, 'id': 'call_3199a256de3849918fb01b', 'type': 'tool_call'}]


### 生成式大语言模型本质就是“文字接龙”,它生成就是一段文字，没有办法调用程序/函数,所以content为空字符串

In [7]:
msg

AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'name': 'multiply', 'arguments': '{"first_int": 5, "second_int": 42}'}, 'index': 0, 'id': 'call_3199a256de3849918fb01b', 'type': 'function'}]}, response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'tool_calls', 'request_id': '2fc5beff-8be4-9a56-9356-2360da5cbdcb', 'token_usage': {'input_tokens': 179, 'output_tokens': 25, 'total_tokens': 204, 'prompt_tokens_details': {'cached_tokens': 0}}}, id='run--56bcd921-bbd0-4699-9e17-89c81ea9a8ab-0', tool_calls=[{'name': 'multiply', 'args': {'first_int': 5, 'second_int': 42}, 'id': 'call_3199a256de3849918fb01b', 'type': 'tool_call'}])

## 最简单的Agent智能体
### 你需要手动提取 tool_call 中的信息，并调用对应的函数来执行任务：

In [9]:
# 假设 msg 是 AIMessage 对象
for tool_call in msg.tool_calls:
    tool_name = tool_call["name"]
    tool_args = tool_call["args"]

    if tool_name == "multiply":
        print(tool_args)
        result = multiply.invoke(tool_args)
        print(f"调用工具 {tool_name}，结果为: {result}")

{'first_int': 5, 'second_int': 42}
调用工具 multiply，结果为: 210


### 如果你希望 LLM 在某些情况下直接给出回答而不是调用工具，可以尝试修改提示或问题形式，比如：

In [10]:
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_core.tools import tool


@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int


llm = ChatTongyi(model="qwen-turbo")

llm_with_tools = llm.bind_tools([multiply])

msg = llm_with_tools.invoke("5乘以42等于多少?请直接告诉我结果。")

print(msg)

content='5乘以42的结果是210。' additional_kwargs={} response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'stop', 'request_id': '042b0e0e-5d1e-97c1-a2a0-388e9e328bad', 'token_usage': {'input_tokens': 184, 'output_tokens': 11, 'total_tokens': 195, 'prompt_tokens_details': {'cached_tokens': 128}}} id='run--951866ec-5d28-45ae-8011-7db272ef4dc2-0'


### 手动构造参数  

即：不依赖自动解析，而是由开发者或用户自行创建和填充调用工具所需的参数。这种方式通常用于调试、自定义逻辑或在某些框架功能受限时使用。

In [11]:
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_core.messages import HumanMessage, SystemMessage

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_time",
            "description": "当你想知道现在的时间时非常有用。",
            "parameters": {},
        },
    },
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "当你想查询指定城市的天气时非常有用。",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "城市或县区，比如北京市、杭州市、余杭区等。",
                    }
                },
            },
            "required": ["location"],
        },
    },
]
# 模拟工具函数的实际逻辑
def get_current_weather(location: str):
    # 这里你可以替换成真实的 API 调用
    return f"{location} 的天气是 18°C，晴天。"

def get_current_time():
    import datetime
    return str(datetime.datetime.now())
    
messages = [
    SystemMessage(content="你是一位得力的助手。"),
    HumanMessage(content="天津的天气怎么样?"),
]
chatLLM = ChatTongyi()
llm_kwargs = {"tools": tools, "result_format": "message"}
ai_message = chatLLM.bind(**llm_kwargs).invoke(messages)
ai_message

AIMessage(content='', additional_kwargs={'tool_calls': [{'function': {'name': 'get_current_weather', 'arguments': '{"location": "天津市"}'}, 'index': 0, 'id': 'call_469a2ee69a2c48fe9fb0f2', 'type': 'function'}]}, response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'tool_calls', 'request_id': '212ce296-598f-9bfb-a92f-c054bb782531', 'token_usage': {'input_tokens': 227, 'output_tokens': 18, 'total_tokens': 245, 'prompt_tokens_details': {'cached_tokens': 0}}}, id='run--22e63040-e05c-458f-be30-d27d8fb6b4eb-0', tool_calls=[{'name': 'get_current_weather', 'args': {'location': '天津市'}, 'id': 'call_469a2ee69a2c48fe9fb0f2', 'type': 'tool_call'}])

In [12]:
# 解析 tool_call
if hasattr(ai_message, "tool_calls") and ai_message.tool_calls:
    for tool_call in ai_message.tool_calls:
        tool_name = tool_call["name"]
        tool_args = tool_call["args"]
    
        print(f"调用工具: {tool_name}，参数为: {tool_args}")
    
        # 手动调用实际函数
        if tool_name == "get_current_weather":
            result = get_current_weather(**tool_args)
            print("天气结果:", result)
        elif tool_name == "get_current_time":
            result = get_current_time()
            print("当前时间:", result)
else:
    print("未调用任何工具，模型直接回复：", ai_message.content)

调用工具: get_current_weather，参数为: {'location': '天津市'}
天气结果: 天津市 的天气是 18°C，晴天。


### 部分模式（Partial Mode） 
该模式允许你从提供的初始文本开始，启用大模型继续生成后续内容。

In [13]:
from langchain_community.chat_models.tongyi import ChatTongyi
from langchain_core.messages import AIMessage, HumanMessage

messages = [
    HumanMessage(
        content="""请接续“难道你很想天使问我? 我的舞步跳得可美”这句话，表达情感的复杂和作者的伤感之情。"""
    ),
    AIMessage(
        content="为何", additional_kwargs={"partial": True}
    ),
]
chatLLM = ChatTongyi()
ai_message = chatLLM.invoke(messages)
ai_message

AIMessage(content='你的目光总是穿透我，落在那遥不可及的天际？  \n我的舞步跳得可美，却无人为我鼓掌，连回响也显得如此孤独。  \n难道你很想天使问我？可她从未降临，只留下风中空洞的呢喃。  \n\n我的心事如丝线缠绕，挣脱不得，又似落叶飘零，无处安放。  \n那些欢笑与泪水交织的夜晚，如今只剩记忆中的微光，在暗夜中闪烁又熄灭。  \n我多想告诉你，这舞步背后藏着多少无声的痛楚，但话到嘴边，却化作一声叹息。  \n\n或许，我只是你生命里的过客；而我，却将你的影子镌刻在心底，再也无法抹去。', additional_kwargs={}, response_metadata={'model_name': 'qwen-turbo', 'finish_reason': 'stop', 'request_id': '53cc7672-e239-9d12-a5ec-752de7ef4588', 'token_usage': {'input_tokens': 42, 'output_tokens': 167, 'total_tokens': 209, 'prompt_tokens_details': {'cached_tokens': 0}}}, id='run--4287c5c9-e099-42be-a60c-16017015ec8f-0')

## 通义千问与视觉能力 
Qwen-VL（qwen-vl-plus / qwen-vl-max）是支持图像处理的模型。

In [15]:
from langchain_community.chat_models import ChatTongyi
from langchain_core.messages import HumanMessage

chatLLM = ChatTongyi(model_name="qwen-vl-max")
image_message = {
    "image": "https://gitee.com/hellowoody/openharmony-hint/raw/main/assets/imgs/wsl-setting.png",
}
text_message = {
    "text": "描述这张图",
}
message = HumanMessage(content=[text_message, image_message])
res = chatLLM.invoke([message])
res

AIMessage(content=[{'text': '这张图展示了Windows操作系统的“控制面板”中的“程序和功能”设置页面，具体是“启用或关闭 Windows 功能”的窗口。\n\n### 1. 左侧导航栏\n- **控制面板主页**：提供访问控制面板中所有设置的入口。\n- **系统和安全**、**网络和 Internet**、**硬件和声音**、**程序**、**用户账户**、**外观和个性化**、**时钟和区域**、**轻松使用**：这些都是控制面板中的主要分类选项。\n\n### 2. 中间部分\n- **程序和功能**：这是当前选中的选项，显示了与程序管理相关的功能。\n- **卸载程序**：允许用户卸载已安装的程序。\n- **启用或关闭 Windows 功能**：这是当前打开的功能，允许用户启用或禁用特定的Windows功能。\n- **运行行为以前版本的 Windows 编写的程序**：提供兼容性设置，以便运行旧版程序。\n- **默认程序**：允许用户更改媒体或设备的默认设置。\n- **Java**：显示了Java的相关信息。\n\n### 3. 右侧窗口 - 启用或关闭 Windows 功能\n- **标题**：启用或关闭 Windows 功能。\n- **说明**：若要启用一种功能，请选择其复选框。若要关闭一种功能，请清除其复选框。填充的框表示仅启用该功能的一部分。\n\n### 4. 特别标注的部分\n- **黄色圆圈标注**：\n  - **Virtual Machine Platform**（虚拟机平台）：已被选中，表示该功能已启用。\n  - **Windows 虚拟机监控程序平台**：已被选中，表示该功能已启用。\n  - **Windows TIFF IFilter**：未被选中，表示该功能未启用。\n\n- **黄色线条标注**：\n  - **启用或关闭 Windows 功能**：强调了当前打开的功能页面。\n\n### 5. 其他功能选项\n- **.NET Framework 3.5 (包括 .NET 2.0 和 3.0)**、**.NET Framework 4.8 Advanced Services**、**Internet Information Services** 等：这些都是可选的Windows功能，

In [16]:
print(res.content[0]["text"])

这张图展示了Windows操作系统的“控制面板”中的“程序和功能”设置页面，具体是“启用或关闭 Windows 功能”的窗口。

### 1. 左侧导航栏
- **控制面板主页**：提供访问控制面板中所有设置的入口。
- **系统和安全**、**网络和 Internet**、**硬件和声音**、**程序**、**用户账户**、**外观和个性化**、**时钟和区域**、**轻松使用**：这些都是控制面板中的主要分类选项。

### 2. 中间部分
- **程序和功能**：这是当前选中的选项，显示了与程序管理相关的功能。
- **卸载程序**：允许用户卸载已安装的程序。
- **启用或关闭 Windows 功能**：这是当前打开的功能，允许用户启用或禁用特定的Windows功能。
- **运行行为以前版本的 Windows 编写的程序**：提供兼容性设置，以便运行旧版程序。
- **默认程序**：允许用户更改媒体或设备的默认设置。
- **Java**：显示了Java的相关信息。

### 3. 右侧窗口 - 启用或关闭 Windows 功能
- **标题**：启用或关闭 Windows 功能。
- **说明**：若要启用一种功能，请选择其复选框。若要关闭一种功能，请清除其复选框。填充的框表示仅启用该功能的一部分。

### 4. 特别标注的部分
- **黄色圆圈标注**：
  - **Virtual Machine Platform**（虚拟机平台）：已被选中，表示该功能已启用。
  - **Windows 虚拟机监控程序平台**：已被选中，表示该功能已启用。
  - **Windows TIFF IFilter**：未被选中，表示该功能未启用。

- **黄色线条标注**：
  - **启用或关闭 Windows 功能**：强调了当前打开的功能页面。

### 5. 其他功能选项
- **.NET Framework 3.5 (包括 .NET 2.0 和 3.0)**、**.NET Framework 4.8 Advanced Services**、**Internet Information Services** 等：这些都是可选的Windows功能，用户可以根据需要进行启用或禁用。

### 6. 底部按钮
- **确定**：点击以保存所做的更改。
- **取消

In [17]:
chatLLM

ChatTongyi(client=<class 'dashscope.aigc.multimodal_conversation.MultiModalConversation'>, model_name='qwen-vl-max', model_kwargs={}, dashscope_api_key=SecretStr('**********'))